# "Conhecendo a Estrutura do Plotly (Parte 3)"
> "O retorno do rei!"

- toc: true
- branch: master
- badges: false
- comments: true
- author: Augusto dos Santos Pereira
- categories: [Plotly, Gráficos, Python]

In [1]:
# hide_input
# This cell is required for the export to HTML to work.
import plotly.io as pio
#pio.renderers.default = 'plotly_mimetype+notebook'
pio.renderers.default = 'colab'
# Default is plotly_mimetype+notebook, but jekyll fails to parse plotly_mimetype.
#pio.renderers.default = 'notebook_connected'
# Uncomment below to avoid using a CDN for plotly.js
# pio.renderers.default = 'notebook'

# Inject the missing require.js dependency.
from IPython.display import display, HTML
#js = '<script src="https://cdnjs.cloudflare.com/ajax/libs/require.js/2.3.6/require.min.js" integrity="sha512-c3Nl8+7g4LMSTdrm621y7kf9v3SDPnhxLNhcjFJbKECVnmZHTdo+IRO05sNLTH/D3vA6u1X32ehoLC7WFVdheg==" crossorigin="anonymous"></script>'
#display(HTML(js))

## A Trilogia

Esta postagem é o fim de uma trilogia em que procurei apresentar a **estrutura do pacote [Plotly](https://plotly.com/python/)**, a ferramenta do ecossistema Python para dados que mais tenho utilizado na elaboração de produtos analíticos - gráficos, cartogramas e diagramas. 

Até aqui, vimos cinco dos sete subpacotes principais do Plotly. Na **[primeira parte](https://augustogeog.github.io/augustogeog/plotly/gr%C3%A1ficos/python/2021/05/31/introdu%C3%A7%C3%A3o-plotly.html)** da série
vimos que **Plotly Data** apresenta funções para carregamento de datasets-exemplos como Pandas DataFrame, que **Plotly Express** compõe uma interface de alto nível de abstração que permite fazer gráficos arrojados com uma única função, enquanto **Plotly Colors** agrega funções que geram diferentes tipos de paletas de cores cíclicas, divergentes, sequenciais e qualitativas.

Na **[segunda parte](https://augustogeog.github.io/augustogeog/plotly/gr%C3%A1ficos/python/2021/06/05/estrutura-plotly-part2.html)**, vimos que  **Graph Objects**, por sua vez, são objetos que envolvem baixo nível de abstração, permitindo maior nível de personalização de gráficos, ao passo que o pacote **Subplots** é voltado para a facilitação da geração de figuras compostas por diversos gráficos.

Agora, mente aberta, espinha erata e o coração tranquilo, pois vamos ver um pouco do que é possível fazer com Plotly **Plotly IO** e **Figure Factory**.

> Important: A estrutura do pacote e os exemplos de código utilizados nesta série se referem à versão 4.14.3 do Plotly.

## Plotly IO

IO é uma sigla bastante recorrente em tecnologias computacionais. Refere-se a input e output. Nas bibliotecas do ecossistema Python para tecnologias de dados, IO usualmente se refere a importação e exportação de dados. Assim, no subpacote **Plotly IO**, temos diversas funções para importar e exportar dados estruturados para formação de Plotly Figures, com suas instruções para renderização de gráficos, cartogramas e diagramas.

Plotly IO apresenta funções para que possamos exportar os nossos gráficos para formatos de imagem (pio.write_image), para html (pio.write_html) e json (pio.write_json). O subpacote ainda nos permite escolher qual o tema padrão para _template_, ou seja, a aparência geral dos gráficos (`pio.templates.default = 'ggplot2'`, por exemplo, faz com que os gráficos tenha aparência do pacote mais utilizado em linguagem R). No namespace IO podemos também escolher o renderizador a ser utilizado (`pio.renderers.default = 'plotly_mimetype+notebook'`, para Jupyter Notebooks, e `pio.renderers.default = 'colab'` para Google Colab, por exemplo). 

Vejamos adiante como exportar os gráficos em formato JSON e HTML, o que pode ser útil para lançamento de serviços _web_. Como sempre, primeiramente vamos importar as bibliotecas necessárias. Em seguida vamos utilizar a função wind() do subpacote data para obter um Pandas DataFrame com dados sobre direção e velocidade dos ventos. Veremos, também, as primerias seis linhas do DataFrame.

In [5]:
import plotly.io as pio # importando a biblioteca Plotly IO
import pandas as pd # importando pacote pandas para geração de DataFrame para dados tabulares
import plotly.express as px # importando subpacote Plotly Express
from plotly.data import wind

df = wind()

df.head(6)

,direction,strength,frequency
0,N,0-1,0.5
1,NNE,0-1,0.6
2,NE,0-1,0.5
3,ENE,0-1,0.4
4,E,0-1,0.4
5,ESE,0-1,0.3


Agora vamos elaborar um gráfico de **[rosa dos ventos](https://content.meteoblue.com/pt/dimensoes-temporais/history/rosa-dos-ventos)**, com a função bar_polar do Plotly Express. Chamamos a função, indicamos qual será o DataFrame, indicamos que o raio, r, é dado pela frequência dos ventos, que o ângulo theta representará a direção, que a cor deve representar a força dos ventos. 

Uma vez que a coluna força, _strength_, apresenta valor textual no DataFrame, as cores serão atribuídas pelo Plotly para variáveis categórias. Nós sabemos, no entanto, que os textos apresentam intervalos de força (0-1, 1-2, 2-3, 3-4, 4-4, 4-5, 5-6, 6+), que são mais bem representados com cores em paleta sequencial. Por essa razão, vamos indicar que a sequência de cores discretas deverá ser igual à paleta de cores sequenciais em vermelho.

In [7]:
fig = px.bar_polar( # chamando a função bar_olar
    data_frame=df # indicando o df como o DataFrame a ser utilizado para a elaboração da figura
    , r="frequency" # indicando que a frequência deve dar a amplitude da barra no raio
    , theta="direction" # indicando que o ãngulo deve representar a coluan direção
    , color="strength" # associação da força dos ventos à cor no gráfico
    , color_discrete_sequence= px.colors.sequential.Reds # indicando que uma gradação de vermelhos deve estar na sequência de cores a ser utilizada
    , title= 'Velocida e Intensidade dos Ventos' # título do gráfico
)

fig.show() # renderizando a figura

Podemos precisar importar esse gráfico em um ambiente no qual não teremos a possibilidade de utilizar Pandas e Plotly da maneira como fizemos até aqui, por restrição de memória, ou incompatibilidades com as ferramentas em uso, por exemplo. Nesse caso, pode ser aconselhável exportar a figura renderizada para HTML e incorporá-la à página por meio dessa linguagem de marcação. Vejamos como transformar o gráfico em HTML e como esse objeto pode ser renderizado em Jupyter Notebooks.

In [12]:
fig_html = pio.to_html(fig) # transformando a figura em html

Abaixo podemos ver um breve trecho do HTML gerado.

In [12]:
fig_html[:500] # exibindo os 500 primeiros caracteres do HTML

'<html>\n<head><meta charset="utf-8" /></head>\n<body>\n    <div>                        <script type="text/javascript">window.PlotlyConfig = {MathJaxConfig: \'local\'};</script>\n        <script type="text/javascript">/**\n* plotly.js v1.58.4\n* Copyright 2012-2020, Plotly, Inc.\n* All rights reserved.\n* Licensed under the MIT license\n*/\n!function(t){if("object"==typeof exports&&"undefined"!=typeof module)module.exports=t();else if("function"==typeof define&&define.amd)define([],t);else{("undefined"!=typ'

Agora podemos utilizar as funções display e HTML para podermos renderizar o objeto fig_html.

Há muito mais que pode ser feito com PIO, mas aqui o importante é entendermos o que podemos encontrar nesse subpacote. Isso vai nos ajudar na hora que precisarmos importar e exportar dados com Plotly.